In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow import keras 
import matplotlib.pyplot as plt

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("data/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
application_df.isnull().sum()

EIN                       0
NAME                      0
APPLICATION_TYPE          0
AFFILIATION               0
CLASSIFICATION            0
USE_CASE                  0
ORGANIZATION              0
STATUS                    0
INCOME_AMT                0
SPECIAL_CONSIDERATIONS    0
ASK_AMT                   0
IS_SUCCESSFUL             0
dtype: int64

In [3]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

application_df.drop(columns=['EIN', 'NAME'], inplace=True)

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# cutoff = 1000
application_types_to_replace = ['T8', 'T7', 'T10', 'T9', 'T13', 'T12', 'T2', 'T25', 'T14', 'T29', 'T15', 'T17']

In [6]:
# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1236        1
C1900        1
C1245        1
C1248        1
C4200        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
application_df['CLASSIFICATION'].value_counts().loc[lambda x : x>1] 

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1278       10
C1238       10
C1235        9
C1237        9
C7210        7
C1720        6
C4100        6
C2400        6
C1257        5
C1600        5
C0           3
C2710        3
C1260        3
C1234        2
C1267        2
C1256        2
C3200        2
C1246        2
Name: CLASSIFICATION, dtype: int64

In [9]:
application_df['CLASSIFICATION'].unique()

array(['C1000', 'C2000', 'C3000', 'C1200', 'C2700', 'C7000', 'C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C2100', 'C1238',
       'C5000', 'C7120', 'C1800', 'C4100', 'C1400', 'C1270', 'C2300',
       'C8200', 'C1500', 'C7210', 'C1300', 'C1230', 'C1280', 'C1240',
       'C2710', 'C2561', 'C1250', 'C8000', 'C1245', 'C1260', 'C1235',
       'C1720', 'C1257', 'C4500', 'C2400', 'C8210', 'C1600', 'C1278',
       'C1237', 'C4120', 'C2170', 'C1728', 'C1732', 'C2380', 'C1283',
       'C1570', 'C2500', 'C1267', 'C3700', 'C1580', 'C2570', 'C1256',
       'C1236', 'C1234', 'C1246', 'C2190', 'C4200', 'C0', 'C3200',
       'C5200', 'C1370', 'C2600', 'C1248', 'C6100', 'C1820', 'C1900',
       'C2150'], dtype=object)

In [10]:
# CLASSIFICATION binning - create a list of classifications to be replaced - cutoff is now 700 instead of 1000

classifications_to_replace = ['C2700', 'C7200',
       'C1700', 'C4000', 'C7100', 'C2800', 'C6000', 'C1238',
       'C5000', 'C7120', 'C1800', 'C4100', 'C1400', 'C1270', 'C2300',
       'C8200', 'C1500', 'C7210', 'C1300', 'C1230', 'C1280', 'C1240',
       'C2710', 'C2561', 'C1250', 'C8000', 'C1245', 'C1260', 'C1235',
       'C1720', 'C1257', 'C4500', 'C2400', 'C8210', 'C1600', 'C1278',
       'C1237', 'C4120', 'C2170', 'C1728', 'C1732', 'C2380', 'C1283',
       'C1570', 'C2500', 'C1267', 'C3700', 'C1580', 'C2570', 'C1256',
       'C1236', 'C1234', 'C1246', 'C2190', 'C4200', 'C0', 'C3200',
       'C5200', 'C1370', 'C2600', 'C1248', 'C6100', 'C1820', 'C1900',
       'C2150']

In [11]:
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [12]:
#dropping more columns that we dont want 
application_df.drop(columns=['APPLICATION_TYPE', 'SPECIAL_CONSIDERATIONS'], inplace=True)

In [13]:
application_df

,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,Independent,C1000,ProductDev,Association,1,0,5000,1
1,Independent,C2000,Preservation,Co-operative,1,1-9999,108590,1
2,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,6692,1
4,Independent,C1000,Heathcare,Trust,1,100000-499999,142590,1
...,...,...,...,...,...,...,...,...
34294,Independent,C1000,ProductDev,Association,1,0,5000,0
34295,CompanySponsored,C3000,ProductDev,Association,1,0,5000,0
34296,CompanySponsored,C2000,Preservation,Association,1,0,5000,0
34297,Independent,C3000,ProductDev,Association,1,0,5000,1


In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
application_df = pd.get_dummies(application_df, columns=['AFFILIATION', 'CLASSIFICATION', 'USE_CASE', 'ORGANIZATION', 'INCOME_AMT'], prefix='DMMY')

In [15]:
application_df

,STATUS,ASK_AMT,IS_SUCCESSFUL,DMMY_CompanySponsored,DMMY_Family/Parent,DMMY_Independent,DMMY_National,DMMY_Other,DMMY_Regional,DMMY_C1000,...,DMMY_Trust,DMMY_0,DMMY_1-9999,DMMY_10000-24999,DMMY_100000-499999,DMMY_10M-50M,DMMY_1M-5M,DMMY_25000-99999,DMMY_50M+,DMMY_5M-10M
0,1,5000,1,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
1,1,108590,1,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
3,1,6692,1,1,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
4,1,142590,1,0,0,1,0,0,0,1,...,1,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,1,0,0,0,1,...,0,1,0,0,0,0,0,0,0,0
34295,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34296,1,5000,0,1,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
34297,1,5000,1,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0


In [16]:
application_df.columns

Index(['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'DMMY_CompanySponsored',
       'DMMY_Family/Parent', 'DMMY_Independent', 'DMMY_National', 'DMMY_Other',
       'DMMY_Regional', 'DMMY_C1000', 'DMMY_C1200', 'DMMY_C2000', 'DMMY_C2100',
       'DMMY_C3000', 'DMMY_C7000', 'DMMY_Other', 'DMMY_CommunityServ',
       'DMMY_Heathcare', 'DMMY_Other', 'DMMY_Preservation', 'DMMY_ProductDev',
       'DMMY_Association', 'DMMY_Co-operative', 'DMMY_Corporation',
       'DMMY_Trust', 'DMMY_0', 'DMMY_1-9999', 'DMMY_10000-24999',
       'DMMY_100000-499999', 'DMMY_10M-50M', 'DMMY_1M-5M', 'DMMY_25000-99999',
       'DMMY_50M+', 'DMMY_5M-10M'],
      dtype='object')

In [18]:
# Split our preprocessed data into our features and target arrays
#  YOUR CODE GOES HERE
y = application_df['IS_SUCCESSFUL']
X = application_df[['STATUS', 'ASK_AMT', 'IS_SUCCESSFUL', 'DMMY_CompanySponsored',
       'DMMY_Family/Parent', 'DMMY_Independent', 'DMMY_National', 'DMMY_Other',
       'DMMY_Regional', 'DMMY_C1000', 'DMMY_C1200', 'DMMY_C2000', 'DMMY_C2100',
       'DMMY_C3000', 'DMMY_C7000', 'DMMY_Other', 'DMMY_CommunityServ',
       'DMMY_Heathcare', 'DMMY_Other', 'DMMY_Preservation', 'DMMY_ProductDev',
       'DMMY_Association', 'DMMY_Co-operative', 'DMMY_Corporation',
       'DMMY_Trust', 'DMMY_0', 'DMMY_1-9999', 'DMMY_10000-24999',
       'DMMY_100000-499999', 'DMMY_10M-50M', 'DMMY_1M-5M', 'DMMY_25000-99999',
       'DMMY_50M+', 'DMMY_5M-10M']]
# Split the preprocessed data into a training and testing dataset
#  YOUR CODE GOES HERE
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


In [19]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [20]:
X_train.shape

(25724, 40)

In [21]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

nn = tf.keras.models.Sequential()

# First dense layer (input layer)
nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=40))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))


# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                3280      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 5,741
Trainable params: 5,741
Non-trainable params: 0
_________________________________________________________________


In [22]:
#Create a callback that saves the model's weights every five epochs.
saving_callback = keras.callbacks.ModelCheckpoint('checkpoints/second_try', save_weights_only=True, save_freq=5)
    
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[saving_callback]) 

Epoch 1/100
804/804 [==============================] - 16s 19ms/step - loss: 0.0629 - accuracy: 0.9832
Epoch 2/100
804/804 [==============================] - 15s 18ms/step - loss: 0.0063 - accuracy: 0.9991
Epoch 3/100
804/804 [==============================] - 16s 19ms/step - loss: 0.0024 - accuracy: 0.9998
Epoch 4/100
804/804 [==============================] - 15s 18ms/step - loss: 0.0013 - accuracy: 0.9998
Epoch 5/100
804/804 [==============================] - 15s 19ms/step - loss: 0.0021 - accuracy: 0.9997
Epoch 6/100
804/804 [==============================] - 15s 19ms/step - loss: 4.9743e-04 - accuracy: 0.9999
Epoch 7/100
804/804 [==============================] - 15s 18ms/step - loss: 6.7810e-04 - accuracy: 0.9999
Epoch 8/100
804/804 [==============================] - 14s 18ms/step - loss: 9.7806e-05 - accuracy: 1.0000
Epoch 9/100
804/804 [==============================] - 14s 18ms/step - loss: 4.0686e-04 - accuracy: 0.9999
Epoch 10/100
804/804 [==============================] - 1

804/804 [==============================] - 15s 19ms/step - loss: 2.3218e-10 - accuracy: 1.0000
Epoch 78/100
804/804 [==============================] - 15s 19ms/step - loss: 2.1785e-10 - accuracy: 1.0000
Epoch 79/100
804/804 [==============================] - 15s 18ms/step - loss: 2.0330e-10 - accuracy: 1.0000
Epoch 80/100
804/804 [==============================] - 15s 18ms/step - loss: 1.9005e-10 - accuracy: 1.0000
Epoch 81/100
804/804 [==============================] - 15s 19ms/step - loss: 1.7855e-10 - accuracy: 1.0000
Epoch 82/100
804/804 [==============================] - 15s 19ms/step - loss: 1.6637e-10 - accuracy: 1.0000
Epoch 83/100
804/804 [==============================] - 15s 18ms/step - loss: 1.5929e-10 - accuracy: 1.0000
Epoch 84/100
804/804 [==============================] - 15s 19ms/step - loss: 1.5494e-10 - accuracy: 1.0000
Epoch 85/100
804/804 [==============================] - 15s 19ms/step - loss: 1.5264e-10 - accuracy: 1.0000
Epoch 86/100
804/804 [===================

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - loss: 1.0778e-09 - accuracy: 1.0000 - 2s/epoch - 6ms/step
Loss: 1.0778494763385993e-09, Accuracy: 1.0


In [25]:
# Export our model to HDF5 file
nn.save('models/AlphabetSoupCharity_optimization.h5')
#  YOUR CODE GOES HERE